In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

print("OpenCV Version:", cv2.__version__)
print("NumPy Version:", np.__version__)

print("Ultralytics YOLO Imported Successfully!")


In [ ]:
# Method 1 CV2
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# Load YOLO model
model = YOLO("yolov8n.pt")

# Initialize video capture (0 for webcam, or replace with video file path)
cap = cv2.VideoCapture(0)

# Store detected vehicles (ID -> (x, y, type))
tracked_vehicles = {}
vehicle_id_counter = 0

# Active vehicle count per type
active_vehicles = defaultdict(int)

def get_vehicle_counts():
    """Returns the current count of detected vehicles dynamically."""
    return {k: v for k, v in active_vehicles.items() if v > 0}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO object detection
    results = model(frame)

    # Temporary storage for new vehicle positions
    new_tracked_vehicles = {}

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            class_id = int(box.cls[0])
            class_name = model.names[class_id]

            # Only consider vehicle types
            if class_name in {"car", "truck", "bus", "motorcycle"}:

                # Check if this vehicle is already tracked
                assigned_id = next(
                    (vid for vid, (px, py, v_type) in tracked_vehicles.items()
                     if abs(center_x - px) < 50 and abs(center_y - py) < 50), 
                    None
                )

                # Assign new ID if it's a new vehicle
                if assigned_id is None:
                    assigned_id = vehicle_id_counter
                    vehicle_id_counter += 1
                    active_vehicles[class_name] += 1

                # Store new position
                new_tracked_vehicles[assigned_id] = (center_x, center_y, class_name)

                # Draw bounding box & label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{class_name} {assigned_id}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                cv2.circle(frame, (center_x, center_y), 5, (255, 255, 255), -1)

    # Update vehicle counts for exiting vehicles
    for vid, (_, _, v_type) in tracked_vehicles.items():
        if vid not in new_tracked_vehicles:
            active_vehicles[v_type] = max(0, active_vehicles[v_type] - 1)

    # Update tracked vehicles
    tracked_vehicles = new_tracked_vehicles

    # Display vehicle counts on screen
    for i, (v_type, count) in enumerate(get_vehicle_counts().items()):
        cv2.putText(frame, f"{v_type}: {count}", (30, 50 + i * 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show frame with detections
    cv2.imshow("Vehicle Detection", frame)

    # Print the latest vehicle count dynamically
    print(get_vehicle_counts())

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Final count after video ends
print("Final Count:", get_vehicle_counts())


In [ ]:
# Method 2 import cv2 & YOLO
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

# Load YOLOv8 model
model = YOLO("yolov8n.pt")

# Initialize video capture (0 = webcam or provide a file path)
cap = cv2.VideoCapture(0)

# Vehicle tracking data
tracked_vehicles = {}
vehicle_id_counter = 0
active_vehicles = defaultdict(int)

# Acceptable vehicle classes
VALID_VEHICLES = {"car", "truck", "bus", "motorcycle"}

def get_vehicle_counts():
    """Return active vehicle counts by type."""
    return {k: v for k, v in active_vehicles.items() if v > 0}

def assign_vehicle_id(cx, cy, vtype):
    """Assign or reuse a vehicle ID based on position proximity."""
    global vehicle_id_counter
    for vid, (px, py, ptype) in tracked_vehicles.items():
        if abs(cx - px) < 50 and abs(cy - py) < 50 and vtype == ptype:
            return vid
    # New vehicle
    vid = vehicle_id_counter
    vehicle_id_counter += 1
    active_vehicles[vtype] += 1
    return vid

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Optional: Resize frame for performance
    # frame = cv2.resize(frame, (640, 480))

    results = model(frame)
    new_tracked_vehicles = {}

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            class_id = int(box.cls[0])
            class_name = model.names[class_id]

            if class_name not in VALID_VEHICLES:
                continue

            vehicle_id = assign_vehicle_id(cx, cy, class_name)
            new_tracked_vehicles[vehicle_id] = (cx, cy, class_name)

            # Draw bounding box, ID, and center point
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name} {vehicle_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 5, (255, 255, 255), -1)

    # Handle vehicles that left the frame
    for vid, (_, _, vtype) in tracked_vehicles.items():
        if vid not in new_tracked_vehicles:
            active_vehicles[vtype] = max(0, active_vehicles[vtype] - 1)

    # Update tracked vehicles
    tracked_vehicles = new_tracked_vehicles

    # Display vehicle counts
    for i, (vtype, count) in enumerate(get_vehicle_counts().items()):
        cv2.putText(frame, f"{vtype}: {count}", (30, 50 + i * 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show frame
    cv2.imshow("Vehicle Detection", frame)

    # Debug print
    print(get_vehicle_counts())

    # Exit key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap.release()
cv2.destroyAllWindows()

print("Final Count:", get_vehicle_counts())


In [ ]:
import sys
sys.path.append('F:/1 Projects/Car Detect/sort')
from sort.sort import Sort


In [ ]:
# Method 3 import cv2,SORT & YOLO
import cv2
import numpy as np
from ultralytics import YOLO
from sort.sort import Sort  # Adjust import based on your actual folder structure
from collections import defaultdict

# Load YOLOv8 model (make sure the model file is in your working directory or provide full path)
model = YOLO("yolov8n.pt")

# Initialize video capture (0 for webcam, or provide a video file path)
cap = cv2.VideoCapture(0)

# Initialize SORT tracker with reasonable parameters
tracker = Sort(max_age=30, min_hits=3, iou_threshold=0.3)

# Valid vehicle classes to detect and track
VALID_VEHICLES = {"car", "truck", "bus", "motorcycle"}

# Store current active counts of vehicles by type
active_vehicles = defaultdict(int)

# Map SORT track IDs to their vehicle class (for counting and display)
id_to_class = {}

def iou(boxA, boxB):
    """Calculate Intersection over Union (IOU) between two bounding boxes.
    Boxes are in format [x1, y1, x2, y2].
    """
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    unionArea = boxAArea + boxBArea - interArea
    if unionArea == 0:
        return 0
    return interArea / unionArea

def get_vehicle_counts():
    """Return current counts of active vehicles by class."""
    return {k: v for k, v in active_vehicles.items() if v > 0}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for consistent processing size and speed
    frame = cv2.resize(frame, (640, 480))

    # Run detection on frame
    results = model(frame)[0]  # Take first (and only) image result

    detections = []
    detection_classes = []

    # Extract detected bounding boxes, confidence scores, and class names
    for box in results.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # Convert tensor to numpy array
        score = box.conf[0].item()
        class_id = int(box.cls[0])
        class_name = model.names[class_id]

        if class_name not in VALID_VEHICLES:
            continue
        if score < 0.3:  # Confidence threshold
            continue

        detections.append([x1, y1, x2, y2, score])
        detection_classes.append(class_name)

    detections_np = np.array(detections)

    # Update SORT tracker only if there are detections, else provide empty array
    if detections_np.shape[0] == 0:
        tracked_objects = np.empty((0, 5))  # No tracked objects
    else:
        tracked_objects = tracker.update(detections_np)

    # Clear counts, will re-compute this frame
    active_vehicles.clear()

    # Associate tracked objects to detected classes using IoU for better matching
    for track in tracked_objects:
        x1, y1, x2, y2, track_id = track.astype(int)
        track_bbox = [x1, y1, x2, y2]

        # Matching detection bbox with highest IoU to this track bbox
        best_iou = 0
        matched_class = None
        for det_bbox, det_class in zip(detections, detection_classes):
            iou_score = iou(track_bbox, det_bbox[:4])
            if iou_score > best_iou:
                best_iou = iou_score
                matched_class = det_class

        if matched_class is None:
            matched_class = "unknown"

        # Update vehicle class mapping and count
        id_to_class[track_id] = matched_class
        active_vehicles[matched_class] += 1

        # Draw bounding box and label on frame
        color = (0, 255, 0)  # Green box
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{matched_class} ID:{track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        # Draw center point circle
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.circle(frame, (cx, cy), 5, (255, 255, 255), -1)

    # Display counts for each vehicle type on the frame
    for i, (vtype, count) in enumerate(get_vehicle_counts().items()):
        cv2.putText(frame, f"{vtype}: {count}", (30, 50 + i * 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    # Show the frame
    cv2.imshow("Vehicle Detection & Tracking", frame)

    # Print counts in console (can be removed if verbose output is not desired)
    print(get_vehicle_counts())

    # Quit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

print("Final Count:", get_vehicle_counts())
